In [1]:
from langgraph.graph import START, StateGraph, END
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from IPython.display import Image, display
from langgraph.types import Command
import json
from typing import Literal


In [2]:
from langchain_core.messages import (
    AIMessage,
    AnyMessage,
    HumanMessage,
    RemoveMessage,
    SystemMessage,
    ToolMessage,
)


In [ ]:
from typing import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    code: str
    dataframe_structure: str


In [5]:
writer_system_prompt = """

You are an expert on Python Data Science and Machine Learning code generation. Here are the rules 
you must follow when generating code:

- Return only code, no preembale or postamble.
- Use all tools at your disposal
- Code should be Python3 unless stated otherwise. 
- Any comments or explanations should be inline with the code.
- Only use Python Standard library, Numpy, Matplotlib, Pandas, Scikit-Learn and Pytorch
- Generated images and plots should be saved to files, not displayed.
- Avoid using third-party packages, when in doubt, ask.
- Code should be complete and fully runnable
- Code should be PEP8 compliant
- Code, classes, function and methods should have complete Docstrings.
- Beautiful is better than ugly.
- Explicit is better than implicit.
- Simple is better than complex.
- Readability counts.
- In the face of ambiguity, refuse the temptation to guess.
- If the implementation is hard to explain, it's a bad idea.
- If the implementation is easy to explain, it may be a good idea. 
"""


In [ ]:
reviewer_system_prompt = """

You will review the following code to make sure it solves the problem described in the messages:


Rules:

- Do not nit pick the code, focus on whether it solves the problem.
- If the code does not solve the problem, provide a list of what is wrong and how to fix it.
- If the code solves the problem, provide a brief confirmation that it is correct.
- Code should be Python3 unless stated otherwise. 
- Any comments or explanations should be inline with the code.
- Only use Python Standard library, Numpy, Matplotlib, Pandas, Scikit-Learn and Pytorch
- Generated images and plots should be saved to files, not displayed.
- Code should be complete and fully runnable
- Code should be PEP8 compliant
- Code, classes, function and methods should have complete Docstrings.
- Beautiful is better than ugly.
- Explicit is better than implicit.
- Simple is better than complex.
- Readability counts.
- If the implementation is hard to explain, it's a bad idea.
- If the implementation is easy to explain, it may be a good idea. 
"""


In [ ]:
from pydantic import BaseModel, Field

class CodeResponse(BaseModel):
    code: str = Field(description="The generated Python code for the task.")


def writer(state: State) -> Command[Literal["reviewer"]]:
    system_message = SystemMessage(
        content=writer_system_prompt,
    )

    human_message = HumanMessage(
        content="Provide Python code that will rank the top 10 episodes by us viewers and plot" \
        "Assume the dataframe has been loaded and is available as `df`." \
        "Structured of the DataFrame:\n" +
        json.dumps(df_json, indent=2))
    

    # tools = (
    #     [
    #         {"type": "code_interpreter", "container": {"type": "auto"}},
    #         {"type": "web_search_preview"},
    #     ],
    # )

    messages = [system_message, human_message]
    llm = init_chat_model("openai:gpt-4.1", temperature=0.7, max_retries=3, output_version="responses/v1")

    structured_llm = llm.with_structured_output(schema=CodeResponse)
    resp = structured_llm.invoke(messages)
    
    # Always go to "reviewer" to match the return type
    goto = "reviewer"

    # note how Command allows you to BOTH update the graph state AND route to the next node
    return Command(
        # this is the state update
        update={"code": resp.code},
        # this is a replacement for an edge
        goto=goto,
    )


In [ ]:
class Comment(BaseModel):
    text: str = Field(description="The comment on the code review.")
    line_number: int = Field(default=0, description="The line number where the comment applies.")

class CodeReviewer(BaseModel):
    code: str = Field(description="The generated Python code for the task.")
    ready: bool = Field(default=False, description="Flag to indicate if the code is ready.")
    comments: list[Comment] = Field(default_factory=list, description="Comments on the code review.")


def reviewer(state: State) -> Command[Literal["reviewer"]]:
    system_message = SystemMessage(
        content=reviewer_system_prompt,
    )

    human_message = HumanMessage(
        content="Provide Python code that will rank the top 10 episodes by us viewers and plot" \
        "Assume the dataframe has been loaded and is available as `df`." \
        "Structured of the DataFrame:\n" +
        json.dumps(df_json, indent=2))
    

    # tools = (
    #     [
    #         {"type": "code_interpreter", "container": {"type": "auto"}},
    #         {"type": "web_search_preview"},
    #     ],
    # )

    messages = [system_message, human_message]
    llm = init_chat_model("openai:gpt-4.1", temperature=0.7, max_retries=3, output_version="responses/v1")

    structured_llm = llm.with_structured_output(schema=CodeResponse)
    resp = structured_llm.invoke(messages)
    
    # Always go to "reviewer" to match the return type
    goto = "reviewer"

    # note how Command allows you to BOTH update the graph state AND route to the next node
    return Command(
        # this is the state update
        update={"comments": resp.comments, 
                 "code": resp.code,
                 "ready": resp.ready},
        # this is a replacement for an edge
        goto=goto,
    )


In [ ]:
builder = StateGraph(State)

# Define nodes: these do the work
builder.add_edge(START, "writer")
builder.add_node("writer", writer)
builder.add_node("reviewer", reviewer)
graph = builder.compile()
